# NW Summer 25 

### Asset Recommender System

The code below builds out a content-based recommender system. In order to promote quick decision making with a small number of users, a KNN-style approach where top stocks are featurized and used in an instance-based way.

Intake user preferences (possibly from a form), such as asset industry [sector], risk tolerance (growth vs. value), desired PE ratio, market-cap level (low, medium, or high), EPS, dividend level (low or none, medium, or high). Then, perform a KNN-style query of the pre-engineered database of [asset] instances to find the top n best fitting assets.  

### 1: Dependency installation & imports

In [1]:
!pip install yfinance
!pip install pandas
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 100.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.4/181.4 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.3/187.3 kB 34.7 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.18.1-cp39-cp39-linux_x86_64.whl size=976150 sha256=61f3017d4ba544eefe9a32a70c13f4ececca7e84e3293ad87cf57b4748303e56
  Stored in directory: /root/.cache/pip/wheels/e7/09/87/1f1a87eb25af98d0a1f60ef184a9779d81c6a2e8d28bc67d7f
Successfully built peewee

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run:

In [2]:
import yfinance as yf 
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

### 2: Retrieving instance data for the model

Retrieve raw data for industry, PE, EPS, market cap, dividend, and risk (growth vs value) for a large number of popular assets. For a well-rounded and diverse set of data, pull the assets from the Russell 3000. 

In [3]:
# first, just testing out the timing to see if it's unreasonable to simply use
# yf.ticker for all the tickers we want to take into consideration

tickers = [
    # Dow Jones Industrial Average (30 companies)
    "MMM", "AXP", "AMGN", "AAPL", "BA", "CAT", "CVX", "CSCO", "KO", "GS",
    "HD", "HON", "IBM", "JNJ", "JPM", "MCD", "MRK", "MSFT", "NKE", "NVDA",
    "PG", "CRM", "TRV", "UNH", "VZ", "V", "WBA", "WMT", "DIS", "DOW",

    # S&P 500 (examples beyond Dow, aiming for diversity)
    "GOOGL", "AMZN", "TSLA", "FB", "BRK.B", "JPM", "V", "JNJ", "WMT", "PG",
    "XOM", "UNH", "HD", "MA", "BAC", "VZ", "PFE", "DIS", "ADBE", "CMCSA",
    "NFLX", "PYPL", "INTC", "CSCO", "PEP", "KO", "T", "COST", "ACN", "PM",
    "NOC", "TXN", "AVGO", "CHTR", "QCOM", "CVS", "MO", "NEE", "UPS", "MDLZ",
    "SBUX", "ISRG", "GILD", "UNP", "AMAT", "LOW", "BKNG", "CME", "GSK", "RTX",

    # Nasdaq 100 (tech-heavy, but with other sectors represented)
    "ASML", "ADBE", "AMAT", "AMD", "ANSS", "ATVI", "BIIB", "CDNS", "CHTR", "CMCSA",
    "COST", "CPRT", "CRWD", "CSX", "DASH", "DXCM", "EA", "EXC", "FANG", "FISV",
    "FTNT", "GILD", "GOOG", "HON", "IDXX", "ILMN", "INTU", "JD", "KDP", "KLAC",
    "LCID", "LRCX", "MAR", "MELI", "MNST", "MRVL", "MTCH", "MU", "NTES", "NXPI",
    "ODFL", "OKTA", "ON", "ORLY", "PAYX", "PCAR", "PDD", "PEP", "PYPL", "QCOM"
]

import time
start_time = time.time()
for t in tickers:
    print(yf.Ticker(t).info)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

# output was just under 23 seconds for 100 tickers. for the russell 3000 this would mean about 
# 15 minutes. i want to see if i could just dump all the ticker info into a csv and then 
# use it quickly via pandas

HTTP Error 401: 
HTTP Error 401: 
{'language': 'en-US', 'region': 'US', 'quoteType': 'EQUITY', 'typeDisp': 'Equity', 'quoteSourceName': 'Nasdaq Real Time Price', 'triggerable': True, 'customPriceAlertConfidence': 'HIGH', 'currency': 'USD', 'regularMarketTime': 1749743638, 'exchange': 'NYQ', 'messageBoardId': 'finmb_289194', 'exchangeTimezoneName': 'America/New_York', 'exchangeTimezoneShortName': 'EDT', 'gmtOffSetMilliseconds': -14400000, 'market': 'us_market', 'esgPopulated': False, 'regularMarketChange': -2.3599854, 'regularMarketDayHigh': 147.27, 'regularMarketDayRange': '144.71 - 147.27', 'regularMarketDayLow': 144.71, 'regularMarketVolume': 739019, 'regularMarketPreviousClose': 147.18, 'bid': 144.85, 'ask': 144.94, 'bidSize': 2, 'askSize': 2, 'fullExchangeName': 'NYSE', 'financialCurrency': 'USD', 'regularMarketOpen': 146.42, 'averageDailyVolume3Month': 3727676, 'averageDailyVolume10Day': 3077710, 'fiftyTwoWeekLowChange': 46.560005, 'fiftyTwoWeekLowChangePercent': 0.47384495, 'fift

Since the time requirement for a few thousand API calls isn't too bad, just make a list of all the stocks in the Russell 3000 and then call yf.Ticker with each one. To obtain the list of Russell tickers (a fairly recent list), I'm using a dataset from GitHub. 

In [4]:
kaggle_df = pd.read_csv("https://raw.githubusercontent.com/bfaure/StockSpider/refs/heads/master/iShares-Russell-3000-ETF_fund.csv", skiprows=8)
kaggle_df.head()

,Ticker,Name,Asset Class,Weight (%),Price,Shares,Market Value,Notional Value,Sector,SEDOL,ISIN,Exchange
0,AAPL,APPLE INC,Equity,3.72,229.28,"1,462,169.00","335,246,108.32","335,246,108.32",Information Technology,2046251,US0378331005,NASDAQ
1,MSFT,MICROSOFT CORP,Equity,2.93,115.15,"2,292,554.00","263,987,593.10","263,987,593.10",Information Technology,2588173,US5949181045,NASDAQ
2,AMZN,AMAZON COM INC,Equity,2.69,"1,971.31","122,971.00","242,413,962.01","242,413,962.01",Consumer Discretionary,2000019,US0231351067,NASDAQ
3,BRKB,BERKSHIRE HATHAWAY INC CLASS B,Equity,1.40,215.29,"585,969.00","126,153,266.01","126,153,266.01",Financials,2073390,US0846707026,New York Stock Exchange Inc.
4,JPM,JPMORGAN CHASE & CO,Equity,1.28,113.97,"1,011,631.00","115,295,585.07","115,295,585.07",Financials,2190385,US46625H1005,New York Stock Exchange Inc.


In [5]:
russell_tickers = kaggle_df["Ticker"].tolist()

In [6]:
russell_tickers

['AAPL',
 'MSFT',
 'AMZN',
 'BRKB',
 'JPM',
 'FB',
 'JNJ',
 'XOM',
 'GOOG',
 'GOOGL',
 'BAC',
 'V',
 'UNH',
 'PFE',
 'T',
 'CVX',
 'HD',
 'CSCO',
 'WFC',
 'VZ',
 'INTC',
 'PG',
 'BA',
 'MA',
 'MRK',
 'C',
 'KO',
 'DIS',
 'NVDA',
 'CMCSA',
 'NFLX',
 'PEP',
 'DWDP',
 'ABBV',
 'IBM',
 'ORCL',
 'WMT',
 'AMGN',
 'MDT',
 'ADBE',
 'PM',
 'MCD',
 'ABT',
 'HON',
 'MMM',
 'UNP',
 'MO',
 'ACN',
 'CRM',
 'QCOM',
 'AVGO',
 'TXN',
 'GE',
 'UTX',
 'NKE',
 'COST',
 'PYPL',
 'LLY',
 'BMY',
 'GILD',
 'TMO',
 'BKNG',
 'LOW',
 'COP',
 'CAT',
 'LMT',
 'SLB',
 'USB',
 'UPS',
 'CVS',
 'GS',
 'NEE',
 'AXP',
 'EOG',
 'SBUX',
 'ANTM',
 'BDX',
 'BIIB',
 'TJX',
 'DHR',
 'ADP',
 'AGN',
 'AET',
 'CELG',
 'ISRG',
 'OXY',
 'PNC',
 'AMT',
 'ATVI',
 'MDLZ',
 'WBA',
 'CB',
 'CSX',
 'SYK',
 'SCHW',
 'RTN',
 'CME',
 'FDX',
 'BLK',
 'CL',
 'MS',
 'DUK',
 'CHTR',
 'INTU',
 'SPG',
 'BSX',
 'ESRX',
 'ILMN',
 'GD',
 'MU',
 'NOC',
 'NSC',
 'DE',
 'VLO',
 'CI',
 'SPGI',
 'VRTX',
 'EMR',
 'PX',
 'FOXA',
 'ITW',
 'PSX',
 'BK',
 'A

In [7]:
ticker = tickers[0]
yticker = yf.Ticker(ticker)
yticker.info.keys()

HTTP Error 401: 


dict_keys(['language', 'region', 'quoteType', 'typeDisp', 'quoteSourceName', 'triggerable', 'customPriceAlertConfidence', 'currency', 'forwardPE', 'priceToBook', 'sourceInterval', 'exchangeDataDelayedBy', 'averageAnalystRating', 'tradeable', 'cryptoTradeable', 'exchange', 'messageBoardId', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'gmtOffSetMilliseconds', 'market', 'esgPopulated', 'corporateActions', 'regularMarketTime', 'marketState', 'hasPrePostMarketData', 'firstTradeDateMilliseconds', 'priceHint', 'regularMarketChange', 'regularMarketDayHigh', 'regularMarketDayRange', 'regularMarketDayLow', 'regularMarketVolume', 'regularMarketPreviousClose', 'bid', 'ask', 'bidSize', 'askSize', 'fullExchangeName', 'financialCurrency', 'regularMarketOpen', 'averageDailyVolume3Month', 'averageDailyVolume10Day', 'fiftyTwoWeekLowChange', 'fiftyTwoWeekLowChangePercent', 'fiftyTwoWeekRange', 'fiftyTwoWeekHighChange', 'fiftyTwoWeekHighChangePercent', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'fift

In [8]:
# let's just time the first 10 to see how long it takes

keep_fields = recommended_info_fields = [
    'longName',
    'sector',
    'industry',
    'country',
    'exchange',
    'marketCap',
    'trailingPE',
    'forwardPE',
    'dividendYield',
    'beta',
    'earningsQuarterlyGrowth',
    'revenueGrowth',
    'profitMargins',
    'returnOnEquity',
    'priceToBook',
    'currentPrice',
    'volume',
    'fiftyTwoWeekChangePercent',
    'averageAnalystRating',
    'recommendationKey',
    'longBusinessSummary'
]

def get_ticker_dict(ticker):
    yticker = yf.Ticker(ticker)
    ticker_dict = {"Ticker": ticker}
    for field in keep_fields:
        ticker_dict[field] = yticker.info.get(field, np.nan)
    return ticker_dict

res = get_ticker_dict("MSFT")

HTTP Error 401: 


In [9]:
pd.DataFrame(res, index=[0])

,Ticker,longName,sector,industry,country,exchange,marketCap,trailingPE,forwardPE,dividendYield,...,revenueGrowth,profitMargins,returnOnEquity,priceToBook,currentPrice,volume,fiftyTwoWeekChangePercent,averageAnalystRating,recommendationKey,longBusinessSummary
0,MSFT,Microsoft Corporation,NaN,NaN,NaN,NMS,3544801345536,36.82857,31.901672,0.7,...,NaN,NaN,NaN,11.014549,NaN,NaN,7.029307,1.4 - Strong Buy,NaN,NaN


In [10]:
!pip install tqdm
import time
from tqdm.auto import tqdm


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
/root/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
lst = []
for t in tqdm(russell_tickers[2500:3000], desc="Fetching ticker data"):
    lst.append(get_ticker_dict(t))

res = pd.DataFrame(lst)
pd.DataFrame(res).to_csv("batch2500_3000.csv")

Fetching ticker data:  39%|███▉      | 195/500 [00:47<01:21,  3.74it/s]HTTP Error 401: 
Exception ignored from cffi callback <function buffer_callback at 0x7f02a9ae7c10>:
Traceback (most recent call last):
  File "/root/venv/lib/python3.9/site-packages/curl_cffi/curl.py", line 68, in buffer_callback
    @ffi.def_extern()
KeyboardInterrupt: 
Fetching ticker data:  39%|███▉      | 195/500 [00:47<01:14,  4.11it/s]


RequestException: Failed to perform, curl: (23) Failure writing output to destination, passed 13 returned 0. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

### 3: Creating & cleaning the final dataset

In [14]:
batch_0 = pd.read_csv("batch0_500.csv")
batch_1 = pd.read_csv("batch500_1000.csv")
batch_2 = pd.read_csv("batch1000_1500.csv")
batch_3 = pd.read_csv("batch1500_2000.csv")
batch_4 = pd.read_csv("batch2000_2500.csv")
batch_5 = pd.read_csv("batch2500_3000.csv")

In [17]:
dataset = pd.concat([batch_0, batch_1, batch_2, batch_3, batch_4, batch_5], ignore_index=True)

In [14]:
dataset.shape[0]

3000

In [15]:
dataset.dropna().shape[0]

614

Looks like there is a lot of missing data, so let's see where those na's are coming from. 

In [16]:

dataset.isna().sum()

Unnamed: 0                      0
Ticker                          0
longName                      850
sector                       1194
industry                     1194
country                      1194
exchange                      838
marketCap                     932
trailingPE                   1451
forwardPE                     998
dividendYield                1748
beta                         1211
earningsQuarterlyGrowth      1747
revenueGrowth                1232
profitMargins                1194
returnOnEquity               1304
priceToBook                   939
currentPrice                 1192
volume                       1174
fiftyTwoWeekChangePercent     928
averageAnalystRating         1698
recommendationKey            1192
longBusinessSummary          1179
dtype: int64

Our instance-based model doesn't need all these fields, so first let's filter out the columns we don't need.

In [17]:
# we want to keep some fields for a growth/value labeling purpose
fields = [
    # value stock features
    'priceToBook',
    'trailingPE',
    'dividendYield',

    # growth stock features
    'revenueGrowth',
    'earningsQuarterlyGrowth'
]
model = dataset[["Ticker", "longName", "sector", "industry", "country", "marketCap", "forwardPE", *fields]]

And now for some data cleaning...

In [18]:
model['dividendYield'] = model['dividendYield'].fillna(0)

/tmp/ipykernel_44/2961092561.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model['dividendYield'] = model['dividendYield'].fillna(0)


In [19]:
model.dropna().shape[0]

1169

Ok, so a large number of our NAs were just companies without a dividend yield - this isn't an issue. However, we see below that a lot of these tickers are not returning any data, for example "FB". This is because some tickers have been changed, ex: "FB" --> "META". 

In [20]:
model.isna().sum()

Ticker                        0
longName                    850
sector                     1194
industry                   1194
country                    1194
marketCap                   932
forwardPE                   998
priceToBook                 939
trailingPE                 1451
dividendYield                 0
revenueGrowth              1232
earningsQuarterlyGrowth    1747
dtype: int64

To address this issue, we'll use AI to create a mapping .csv file for us, mapping the old tickers to the new ones. Then, we'll call the API once again and fill in the rows. 

In [21]:
model[model['longName'].isna()]['Ticker'].to_csv("outdated_tickers.csv", index=False)

In [22]:
model.columns

Index(['Ticker', 'longName', 'sector', 'industry', 'country', 'marketCap',
       'forwardPE', 'priceToBook', 'trailingPE', 'dividendYield',
       'revenueGrowth', 'earningsQuarterlyGrowth'],
      dtype='object')

In [23]:
# this is the mapping we'll use
mapping = {
    'BRKB': 'BRK-B',
    'FB': 'META',
    'DWDP': 'PHM',
    'UTX': 'RTX',
    'ANTM': 'CVH',
    'AGN': 'ABBV',
    'CELG': 'BMY',
    'RTN': 'RTX',
    'BBT': 'PNC',
    'APC': 'APA',
    'WP': 'WRK',
    'CXO': 'COP',
    'ALXN': 'AZN',
    'RHT': 'IBM',
    'CTL': 'LUMN',
    'XLNX': 'AVGO',
    'CBS': 'VIAC',
    'WCG': 'CNC',
    'BLL': 'APD',
    'SYMC': 'CRWD',
    'VIAB': 'PARA',
    'DISCK': 'DISCA',
    'BHGE': 'BKR',
    'HCP': 'PEAK',
    'SIVB': None,   # Delisted (Silicon Valley Bank)
    'LOXO': None,   # Acquired, no ticker
    'CHK': 'OVV',
    'MYL': None,    # Mylan merged into Pfizer Upjohn (ticker changed, no direct mapping)
    'TWTR': None,   # Twitter rebranded to X, ticker still TWTR but company changed
    'SIX': None,    # No current ticker
    'WRE': None,
    'WBT': None,
    'UBSH': None,
    'LSXMA': None,
    'MBFI': None,
    'FCEA': None,
    'GLIBA': None,
    'LSXMK': None,
    'BKI': None,
    'BFB': None,
    'QRTEA': None,
    'STAY': None,
    'JCOM': None,
    'STMP': None,
    'PRSP': None,
    'SRC': None,
    'WMGI': None,
    'UNVR': None,
    'CMD': None,
    'TRCO': None,
    'APY': None,
    'CFX': None,
    'MDR': None,
    'GDI': None,
    'ELLI': None,
    'IMMU': None,
    'CXP': None,
    'JWA': None,
    'TECD': None,
    'AGR': None,
    'MOGA': None,
    'BFA': None,
    'AHL': None,
    'WBT': None,
    'UBSH': None,
    'LEXEA': None,
    'CSFL': None,
    'LGFB': None,
    'GWB': None,
    'ESL': None,
    'MLHR': None,
    'DSW': None,
    'MDP': None,
    'CBM': None,
    'SRCI': None,
    'HSC': None,
    'XLRN': None,
    'ROIC': None,
    'MSFUT': None,
    'PLT': None,
    'AXE': None,
    'CLNC': None,
    'CHSP': None,
    'ADSW': None,
    'GCP': None,
    'BIG': None,
    'ATU': None,
    'BEL': None,
    'SNHY': None,
    'MANT': None,
    'CLDR': None,
    'WDR': None,
    'NGHC': None,
    'RAVN': None,
    'EVBG': None,
    'CWEN US': None,
    'CTB': None,
    'MNR': None,
    'LGFA': None,
    'DPLO': None,
    'WPG': None,
    'BMCH': None,
    'TWNK': None,
    'PJC': None,
    'SWM': None,
    'KNL': None,
    'CISN': None,
}

Update the old tickers and refresh the information...

In [24]:
model.head()

,Ticker,longName,sector,industry,country,marketCap,forwardPE,priceToBook,trailingPE,dividendYield,revenueGrowth,earningsQuarterlyGrowth
0,AAPL,Apple Inc.,Technology,Consumer Electronics,United States,3.014046e+12,24.284006,45.135334,31.384151,0.52,0.051,0.048
1,MSFT,Microsoft Corporation,Technology,Software - Infrastructure,United States,3.479990e+12,31.318394,10.813164,36.155212,0.70,0.133,0.177
2,AMZN,"Amazon.com, Inc.",Consumer Cyclical,Internet Retail,United States,2.292240e+12,35.108128,7.491846,35.165310,0.00,0.086,0.642
3,BRKB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN
4,JPM,JPMorgan Chase & Co.,Financial Services,Banks - Diversified,United States,7.442264e+11,15.997313,2.245887,13.140090,2.10,0.048,0.091


In [25]:
cols_to_update = ['longName', 'sector', 'industry', 'country', 'marketCap',
                      'forwardPE', 'priceToBook', 'trailingPE', 'dividendYield',
                      'revenueGrowth', 'earningsQuarterlyGrowth']

# remove the rows with the outdated tickers:
model = model[~model['Ticker'].isin(mapping.keys())]

# now, retrieve the new data with the updated tickers and add that new data to the dataset
mapping = {v: k for k, v in mapping.items() if v is not None}
for ticker in mapping.keys():
    try:
        yf_ticker = yf.Ticker(ticker)
        info = yf_ticker.info

        new_row = {'Ticker': ticker}
        for col in cols_to_update:
            new_row[col] = info.get(col, None)

        model = pd.concat([model, pd.DataFrame([new_row])], ignore_index=True)
    except:
        print(f'error getting ticker {ticker}')

HTTP Error 401: 
/tmp/ipykernel_44/762056484.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  model = pd.concat([model, pd.DataFrame([new_row])], ignore_index=True)
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 


In [26]:
model[model["Ticker"] == "BRK-B"]

,Ticker,longName,sector,industry,country,marketCap,forwardPE,priceToBook,trailingPE,dividendYield,revenueGrowth,earningsQuarterlyGrowth
2895,BRK-B,Berkshire Hathaway Inc.,None,None,None,1.048392e+12,24.202688,0.001068,12.945924,NaN,NaN,NaN


In [27]:
model.shape[0]

2919

In [28]:
model.isna().sum()

Ticker                        0
longName                    750
sector                     1112
industry                   1112
country                    1112
marketCap                   834
forwardPE                   900
priceToBook                 841
trailingPE                 1356
dividendYield                 9
revenueGrowth              1150
earningsQuarterlyGrowth    1666
dtype: int64

Ok, so we've updated any outdated tickers. We're still seeing a lot of missing data for sector and industry. For our purposes, we can drop these and still retain a good amount of diverse assets. 

In [29]:
model.dropna().shape[0]  # 1268 is enough to work with for a solid asset recommender 

1171

In [30]:
model = model.dropna()

### 4: Data exploration

How many assets do we have in each sector?

In [31]:
model["sector"].value_counts()

sector
Financial Services        264
Industrials               196
Consumer Cyclical         161
Technology                131
Real Estate                94
Healthcare                 87
Consumer Defensive         68
Utilities                  50
Energy                     50
Basic Materials            42
Communication Services     28
Name: count, dtype: int64

In [32]:
model.to_csv("model.csv", index=False)

How many companies offer no dividend? How many high dividend companies are there?

In [33]:
model[model["dividendYield"] == 0].shape[0]

267

In [34]:
model[model["dividendYield"] > 3].shape[0]

338

### 5: Feature engineering

The user is going to provide the following specifications: sector, market cap category (low, mid, or high cap) 

In [35]:
model.columns

Index(['Ticker', 'longName', 'sector', 'industry', 'country', 'marketCap',
       'forwardPE', 'priceToBook', 'trailingPE', 'dividendYield',
       'revenueGrowth', 'earningsQuarterlyGrowth'],
      dtype='object')

Feature 1: market cap level via binning

In [36]:
model['marketCapLevel'] = model['marketCap'].apply(lambda x : 'high' if x > 10e9 else 'mid' if x > 2e9 else 'low')
model['marketCapLevel'].value_counts()

marketCapLevel
high    472
mid     443
low     256
Name: count, dtype: int64

In [37]:
model.head()

,Ticker,longName,sector,industry,country,marketCap,forwardPE,priceToBook,trailingPE,dividendYield,revenueGrowth,earningsQuarterlyGrowth,marketCapLevel
0,AAPL,Apple Inc.,Technology,Consumer Electronics,United States,3.014046e+12,24.284006,45.135334,31.384151,0.52,0.051,0.048,high
1,MSFT,Microsoft Corporation,Technology,Software - Infrastructure,United States,3.479990e+12,31.318394,10.813164,36.155212,0.70,0.133,0.177,high
2,AMZN,"Amazon.com, Inc.",Consumer Cyclical,Internet Retail,United States,2.292240e+12,35.108128,7.491846,35.165310,0.00,0.086,0.642,high
3,JPM,JPMorgan Chase & Co.,Financial Services,Banks - Diversified,United States,7.442264e+11,15.997313,2.245887,13.140090,2.10,0.048,0.091,high
4,JNJ,Johnson & Johnson,Healthcare,Drug Manufacturers - General,United States,3.769108e+11,14.778301,4.824602,17.424917,3.35,0.024,2.379,high


Feature 2: growth vs value classification, taking into account dividend and revenue growth

In [38]:
def classify_gv(stock):
    if stock['dividendYield'] > 3:
        return 'value'
    elif stock['revenueGrowth'] > 0.15 or stock['earningsQuarterlyGrowth'] > 10:
        return 'growth'
    else:
        return 'neutral'

model['growthValueType'] = model.apply(classify_gv, axis=1)

In [39]:
model['growthValueType'].value_counts()

growthValueType
neutral    690
value      338
growth     143
Name: count, dtype: int64

In [40]:
model[model['growthValueType'] == 'growth'].head(15)

,Ticker,longName,sector,industry,country,marketCap,forwardPE,priceToBook,trailingPE,dividendYield,revenueGrowth,earningsQuarterlyGrowth,marketCapLevel,growthValueType
26,NVDA,NVIDIA Corporation,Technology,Semiconductors,United States,3.485841e+12,34.692963,41.575043,46.108067,0.03,0.692,0.262,high,growth
46,QCOM,QUALCOMM Incorporated,Technology,Semiconductors,United States,1.737620e+11,12.939755,6.278145,16.131824,2.29,0.169,0.209,high,growth
47,AVGO,Broadcom Inc.,Technology,Semiconductors,United States,1.148569e+12,39.590760,4.124803,89.151460,0.97,0.164,1.727,high,growth
53,LLY,Eli Lilly and Company,Healthcare,Drug Manufacturers - General,United States,7.191763e+11,35.353043,48.157497,65.130080,0.78,0.452,0.230,high,growth
77,ISRG,"Intuitive Surgical, Inc.",Healthcare,Medical Instruments & Supplies,United States,1.892519e+11,67.178116,11.062645,77.195910,0.00,0.192,0.282,high,growth
87,SCHW,The Charles Schwab Corporation,Financial Services,Capital Markets,United States,1.595572e+11,23.109211,3.955096,26.610607,1.22,0.181,0.402,high,growth
92,MS,Morgan Stanley,Financial Services,Capital Markets,United States,2.108558e+11,16.573770,2.175741,15.426055,2.80,0.163,0.265,high,growth
95,INTU,Intuit Inc.,Technology,Software - Application,United States,2.121392e+11,34.210526,10.545949,62.183160,0.54,0.410,-0.146,high,growth
97,BSX,Boston Scientific Corporation,Healthcare,Medical Devices,United States,1.470721e+11,35.630825,6.620272,72.562050,0.00,0.209,0.362,high,growth
101,MU,"Micron Technology, Inc.",Technology,Semiconductors,United States,1.266877e+11,8.808081,2.605977,27.119620,0.41,0.383,0.996,high,growth


In [41]:
model.to_csv("model.csv", index=False)

In [42]:
model['marketCapLevel'].value_counts()

marketCapLevel
high    472
mid     443
low     256
Name: count, dtype: int64

In [43]:
model.columns

Index(['Ticker', 'longName', 'sector', 'industry', 'country', 'marketCap',
       'forwardPE', 'priceToBook', 'trailingPE', 'dividendYield',
       'revenueGrowth', 'earningsQuarterlyGrowth', 'marketCapLevel',
       'growthValueType'],
      dtype='object')

In [44]:
# retain: Ticker, sector, marketCapLevel, growthValueType
knn_model = model[['Ticker', 'sector', 'marketCapLevel', 'growthValueType', 'forwardPE']]
knn_model = knn_model.set_index('Ticker')

# one hot encode categoricals
categorical_cols = ['sector', 'marketCapLevel', 'growthValueType']
knn_encoded = pd.get_dummies(knn_model, columns=categorical_cols)

In [45]:
knn_encoded.head()

,forwardPE,sector_Basic Materials,sector_Communication Services,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Real Estate,sector_Technology,sector_Utilities,marketCapLevel_high,marketCapLevel_low,marketCapLevel_mid,growthValueType_growth,growthValueType_neutral,growthValueType_value
Ticker,,,,,,,,,,,,,,,,,,
AAPL,24.284006,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False
MSFT,31.318394,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False
AMZN,35.108128,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False
JPM,15.997313,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False
JNJ,14.778301,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True


In [40]:
# fit and retrieve top k nn
user_input = {
    'sector': 'Healthcare',
    'marketCapLevel': 'high',
    'growthValueType': 'growth',
    'forwardPE': 15
}

feature_cols = knn_encoded.columns
X = knn_encoded.values

knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X)

user_df = pd.DataFrame([user_input])
user_encoded = pd.get_dummies(user_df)
user_encoded = user_encoded.reindex(columns=feature_cols, fill_value=0)

distances, indices = knn.kneighbors(user_encoded.values)

nearest_tickers = knn_encoded.iloc[indices[0]].index.tolist()

print("Top 5 similar tickers:", nearest_tickers)


NameError: name 'knn_encoded' is not defined

### 6: One-hot encode the categorical data & update the stored dataset

In [37]:
categorical_cols = ['sector', 'marketCapLevel', 'growthValueType']
knn_encoded = pd.get_dummies(knn_model, columns=categorical_cols)
knn_encoded.to_csv("encoded_model.csv")

NameError: name 'knn_model' is not defined

### 7: Save & load the model

In [31]:
import joblib
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [49]:
encoded_model = pd.read_csv('encoded_model.csv')
X = encoded_model.drop(columns=['Ticker']).values
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X)

# Save
joblib.dump(knn, 'model.joblib')

['model.joblib']

### 8: Create a function for the API endpoint (JSON input & output)

In [61]:
def get_recs(user_input):
    model_data = pd.read_csv('encoded_model.csv')

    # load cached model
    knn = joblib.load('model.joblib')
    user_df = pd.DataFrame([user_input])

    # encode user input
    user_encoded = pd.get_dummies(user_df)
    feature_cols = model_data.columns.drop('Ticker')
    user_encoded = user_encoded.reindex(columns=feature_cols, fill_value=0)

    # retrieve nn
    distances, indices = knn.kneighbors(user_encoded.values)
    nearest_tickers = model_data.iloc[indices[0]].index.tolist()

    print("Top 5 similar tickers:", nearest_tickers)

get_recs({
    'sector': 'Healthcare',
    'marketCapLevel': 'high',
    'growthValueType': 'growth',
    'forwardPE': 15})

Top 5 similar tickers: [122, 30, 4, 96, 195]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b273eb39-632e-4d88-a3fd-8f52123be496' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>